In [32]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [33]:
pd.__version__

'2.2.3'

### Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [39]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
len(df.columns)

19

### Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

In [40]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

std_duration = df['duration'].std()
print(std_duration)


42.594351241920904


### Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [41]:
len(df)

3066766

In [42]:
df_wo_outliers = df[(df.duration >= 1) & (df.duration <= 60)]
len(df_wo_outliers)

3009173

In [43]:
fraction_left = len(df_wo_outliers) * 100 / len(df) 
print(fraction_left)

98.1220282212598


### Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs.  
What's the dimensionality of this matrix (number of columns)?

In [45]:
categorical = ['PULocationID', 'DOLocationID']
df_wo_outliers[categorical] = df_wo_outliers[categorical].astype(str)
train_dicts = df_wo_outliers[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
print(X_train.shape[1])

/tmp/ipykernel_26560/4056254512.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wo_outliers[categorical] = df_wo_outliers[categorical].astype(str)


515


### Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [47]:
target = 'duration'
y_train = df_wo_outliers[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.649261952010433

### Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

In [50]:
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
df_val_wo_outliers = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]
categorical = ['PULocationID', 'DOLocationID']
df_val_wo_outliers[categorical] = df_val_wo_outliers[categorical].astype(str)
val_dicts = df_val_wo_outliers[categorical].to_dict(orient='records')
X_val = dv.fit_transform(val_dicts)
target = 'duration'
y_val = df_val_wo_outliers[target].values
lr.fit(X_val, y_val)
y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

/tmp/ipykernel_26560/3767685214.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val_wo_outliers[categorical] = df_val_wo_outliers[categorical].astype(str)


7.778948516876832